# Introduction

Guided LDA is a method in which the user gives some priors (words) for each or some topics, which is in a way used as a starting point by the algorithm to determine other words in the topics as needed. The package to implement this algorithm is GuidedLDA whos <a href="https://guidedlda.readthedocs.io/en/latest/">user guide</a> explains how to use it on a higher level. However, recently many people across the world are facing challenges in installing the package. Thankfully, someone found a workaround for using the implementation and it is given in detail in this <a href="https://github.com/dex314/GuidedLDA_WorkAround">github</a> repo. if you want to install and try running the notebook for yourself, please do ensure that you follow steps given in the above github link before starting.

## Importing Required packages 

In [3]:
import pandas as pd
import numpy as np
from lda import guidedlda as glda

from sklearn.feature_extraction.text import CountVectorizer

import nltk
#nltk.download()
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

import string

Getting error for lda package? <hr> Getting guided lda package that I am using is not a straightforward exercise as the original packages pip install is not working. I found the workaround for implementing it <a href="https://github.com/dex314/GuidedLDA_WorkAround">here</a>.

## Reading and processing the data

In [13]:
df1=pd.read_csv("tweets.csv")
df1

,Unnamed: 0,text,location,entities
0,0,@55Bellechasse @MLP_officiel @YvesPDB Je pense...,NaN,"{'hashtags': [], 'symbols': [], 'user_mentions..."
1,1,@AmourToujours8 Il doit y avoir méprise. Je vo...,NaN,"{'hashtags': [], 'symbols': [], 'user_mentions..."
2,2,@HeleneLaporteRN @MLP_officiel @RNational_off ...,quelque part,"{'hashtags': [], 'symbols': [], 'user_mentions..."
3,3,@ericwoerth @EmmanuelMacron Non Monsieur Le To...,NaN,"{'hashtags': [], 'symbols': [], 'user_mentions..."
4,4,"@dumontsoniaz Souvenez vous de Kohler, secréta...",NaN,"{'hashtags': [], 'symbols': [], 'user_mentions..."
...,...,...,...,...
995,995,@sebchenu @EmmanuelMacron @FaceaBaba @MLP_offi...,"Gironde, Aquitaine","{'hashtags': [], 'symbols': [], 'user_mentions..."
996,996,#MLP non c'est #Macron #avecvous https://t.co/...,NaN,"{'hashtags': [{'text': 'MLP', 'indices': [0, 4..."
997,997,"""Mais Macron il aura pas le choix d'essayer de...",NaN,"{'hashtags': [], 'symbols': [], 'user_mentions..."
998,998,@ThierryMichels @EmmanuelMacron @avecvousMacro...,"Strasbourg, France",{'hashtags': [{'text': 'jevotemarinelepenle24a...


In [7]:
df1['text'] =df1['text'].str.replace("[^a-zA-Z#]", " ")

stopwords_list = stopwords.words('french')
punctuations = list(set(string.punctuation))

unwanted_list=punctuations+stopwords_list

def clean_text_initial(text):
    text = ' '.join([x.lower() for x in word_tokenize(text) if x.lower() not in unwanted_list and len(x)>1])
    text = ' '.join([x.lower() for x in word_tokenize(text) if nltk.pos_tag([x])[0][1].startswith("NN") or nltk.pos_tag([x])[0][1].startswith("JJ")])
    return text.strip()

df1["clean_text"]=df1.text.apply(lambda text:clean_text_initial(str(text)))
df1.head()

<ipython-input-7-bb103935f5b0>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df1['text'] =df1['text'].str.replace("[^a-zA-Z#]", " ")


,Unnamed: 0,text,location,entities,clean_text
0,0,Bellechasse MLP officiel YvesPDB Je pense...,NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...",bellechasse mlp officiel yvespdb pense sinc re...
1,1,AmourToujours Il doit y avoir m prise Je vo...,NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...",amourtoujours doit avoir prise voterai bien ma...
2,2,HeleneLaporteRN MLP officiel RNational off ...,quelque part,"{'hashtags': [], 'symbols': [], 'user_mentions...",helenelaportern mlp officiel rnational chet pr...
3,3,ericwoerth EmmanuelMacron Non Monsieur Le To...,NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...",ericwoerth emmanuelmacron non monsieur tourne ...
4,4,dumontsoniaz Souvenez vous de Kohler secr ta...,NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...",dumontsoniaz souvenez kohler secr taire ral el...


## Creating objects required for model training 

In [8]:
corpus=df1.clean_text.tolist()
vocab=list(set(word_tokenize(" ".join(df1.clean_text))))
vectorizer = CountVectorizer(ngram_range=(1,1),vocabulary=vocab)
X = vectorizer.fit_transform(corpus)
word2id=vectorizer.vocabulary_

## Defining priors 

In [9]:
health_words=["bedroom","room","house","home","airbnb","condo","bed","blocks","comfy","amenities","pool","gym",
             "cottage","min","minutes", "away","duplex","kitchen","stay","short","apartment","residential",
             "camping","distance","bungalow","walking","neighborhood","cozy","cabin","cabins","coziness",
             "families","property","courtyard","accommodate","living", "area","minutos"]

environment_words=["réchauffement", "climatique", "écologique", "planète"]

immigration_words=["odometer","automobile","car","engine","automatic", "transmissions","manual", "shift",
                      "automotive","chevrolet","transmission","accelerator","toyota","volvo","nissan",
                     "convertibles","convertible","drive"]

education_words=["club","manager","player","championship","contract","stadium","players","season","score","scorer",
              "team","teammate","game","liverpool","football","nfl","victory","ravens","boxing","cricket","quarterback",
              "middleweight","arsenal","barcelona","welterweight","icc","ipl","bowlers","innings","bowler","lightweight",
              "knicks","match","matches","soccer","football","playoffs","premier league","drs","tournament","fan",
              "sports","boxer","fielders","ufc","linebacker","coach","nba","referee","champion","injury","races","points",
              "golf","arenas","pitching","receiver","champ","cornerback","mvp","jayhawks","quarterfinal","agent","ball",
              "comeback","shot","red sox","agency","wins","winners","warriors","gonzaga","race"]

economy_words=["snapchat","facebook","samsung","phone","smartphone","iphone","ai","uber","hewlettpackard","technology",
                ]

society_words=

### Removing prior words that are not part of vocabulary 

In [ ]:
house_words = [x for x in house_words if x in list(word2id.keys())]
glassdoor_words = [x for x in glassdoor_words if x in list(word2id.keys())]
automobile_words = [x for x in automobile_words if x in list(word2id.keys())]
sports_words = [x for x in sports_words if x in list(word2id.keys())]
tech_words = [x for x in tech_words if x in list(word2id.keys())]

### Creating list of word lists as needed 

In [ ]:
seed_topic_list = [
    house_words,
    glassdoor_words,
    automobile_words,
    sports_words,
    tech_words
]

## Defining model 

In [ ]:
model = glda.GuidedLDA(n_topics=5, n_iter=2000, random_state=7, refresh=20,alpha=0.01,eta=0.01)

## Setting priors 

In [ ]:
seed_topics = {}
for t_id, st in enumerate(seed_topic_list):
    for word in st:
        seed_topics[word2id[word]] = t_id

## Training the model 

In [ ]:
model.fit(X, seed_topics=seed_topics, seed_confidence=0.15)

### Seeing the model output topics and top 10 words per topic 

In [ ]:
n_top_words = 10
topic_word = model.topic_word_
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))

## Tagging the topics to create id - topic file

In [ ]:
topic_num_name = {"Topic 0":"room_rentals",
                  "Topic 1":"glassdoor_reviews",
                  "Topic 2":"Automobiles",
                  "Topic 3":"sports_news",
                  "Topic 4":"tech_news"}    

In [ ]:
def get_doc_topics(model_glda,X,num_topics,dataframe,col_name):
    """
    A function which creates dataframe with documents, their dominant topic, along with their probabilities
    
    Parameters
    -------------
    model_glda - Guided LDA trained model
    X - Document term frequency table
    num_topics - Number of topics the model was trained for
    dataframe - Dataframe consisting of cleaned text column
    col_name - Column name in dataframe holding cleaned text
    
    Returns
    -------------
    A dataframe with document number, topic, probability of topic
    """
    df_doc_top = pd.DataFrame()
    final_list = []
    for index in range(len(dataframe[col_name])):
        word_id_dict = dict((x,y) for x,y in zip([x for x in range(num_topics)],np.round(model.transform(X[index])*100,1).tolist()[0]))
        word_score_list = []
        for index in range(num_topics):
            try:
                value = word_id_dict[index]
            except:
                value = 0
            word_score_list.append(value)
        final_list.append(word_score_list)

    df_doc_top = pd.DataFrame(final_list)
    df_doc_top.columns = ['Topic ' + str(i) for i in range(num_topics)]
    df_doc_top.index = ['Document ' + str(i) for i in range(len(dataframe[col_name]))]

    df_doc_top["Dominant_Topic"] = df_doc_top.idxmax(axis=1).tolist()
    df_doc_top["Topic_Probability"] = df_doc_top.max(axis=1).tolist()
    document_df = df_doc_top.reset_index().rename(columns={"index":"Document"})[["Document","Dominant_Topic","Topic_Probability"]]

    return document_df

In [ ]:
document_df=get_doc_topics(model,X,5,df1,"clean_text")

In [ ]:
submission=pd.concat([df1.Id,document_df.Dominant_Topic],axis=1)

In [ ]:
submission.Dominant_Topic=submission.Dominant_Topic.replace(topic_num_name)

In [ ]:
submission=submission.set_index("Id").rename(columns={"Dominant_Topic":"topic"})

In [ ]:
submission